In [1]:
import torch
import torch.nn as nn

In [8]:
class Attention(nn.Module):
    '''
    Experimental implementation of the scaled dot-product attention mechanism
    and not yet fully implemented
    '''
    def __init__(self, embed_size, heads):
        super(Attention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embed size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        energy  = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        # queries shape: (N, query_len, heads, head_dim)
        # keys shape: (N, key_len, heads, head_dim)
        # energy shape: (N, heads, query_len, key_len)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out shape: (N, query_len, heads, head_dim)
        
        # For demonstration purposes, we'll just return the reshaped tensors
        return values, keys, query, energy

In [7]:
# Example usage
# if __name__ == "__main__":
embed_size = 8
heads = 2
batch_size = 1
seq_length = 4

# Create dummy data
values = torch.randn(batch_size, seq_length, embed_size)
keys = torch.randn(batch_size, seq_length, embed_size)
query = torch.randn(batch_size, seq_length, embed_size)

print("Original shapes:")
print(f"Shape of the Values: {values.shape}")
print(f"Shape of the Keys: {keys.shape}")
print(f"Shape of the Query: {query.shape}")
# Create the Attention layer
attention = Attention(embed_size, heads)

# Forward pass
reshaped_values, reshaped_keys, reshaped_query, energy = attention(values, keys, query)

print("\nReshaped shapes:")
print("Reshaped Values shape:", reshaped_values.shape)
print("Reshaped Keys shape:", reshaped_keys.shape)
print("Reshaped Query shape:", reshaped_query.shape)
print(f"Shape of Energy: {energy.shape}")

Original shapes:
Shape of the Values: torch.Size([1, 4, 8])
Shape of the Keys: torch.Size([1, 4, 8])
Shape of the Query: torch.Size([1, 4, 8])


ValueError: einsum(): must specify the equation string and at least one operand, or at least one operand and its subscripts list

In [5]:
import torch
import torch.nn as nn

class Attention(nn.Module):
    '''
    Experimental implementation of the scaled dot-product attention mechanism
    and not yet fully implemented
    '''
    def __init__(self, embed_size, heads):
        super(Attention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embed size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        # Calculate the energy (attention scores)
        energy = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        # queries shape: (N, query_len, heads, head_dim)
        # keys shape: (N, key_len, heads, head_dim)
        # energy shape: (N, heads, query_len, key_len)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Apply softmax to get the attention weights
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        # Multiply the attention weights with the values
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values])
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, head_dim)
        # out shape: (N, query_len, heads, head_dim)

        # Concatenate heads and pass through the final linear layer
        out = out.reshape(N, query_len, self.heads * self.head_dim)
        out = self.fc_out(out)

        return out

# Example usage
if __name__ == "__main__":
    embed_size = 8
    heads = 2
    batch_size = 1
    seq_length = 4

    # Create dummy data
    values = torch.randn(batch_size, seq_length, embed_size)
    keys = torch.randn(batch_size, seq_length, embed_size)
    query = torch.randn(batch_size, seq_length, embed_size)

    print("Original shapes:")
    print(f"Shape of the Values: {values.shape}")
    print(f"Shape of the Keys: {keys.shape}")
    print(f"Shape of the Query: {query.shape}")

    # Create the Attention layer
    attention = Attention(embed_size, heads)

    # Forward pass
    output = attention(values, keys, query)

    print("\nOutput shape:")
    print("Output shape:", output.shape)

Original shapes:
Shape of the Values: torch.Size([1, 4, 8])
Shape of the Keys: torch.Size([1, 4, 8])
Shape of the Query: torch.Size([1, 4, 8])

Output shape:
Output shape: torch.Size([1, 4, 8])
